In [1]:
import numpy as np
import utils
from run_info_utils import get_run_info

In [4]:
experiment_name = 'iccm2019'
run_id = '20190325233424'
output_seq = utils.read_sigmoid_output_seq(experiment_name, run_id)
print(get_run_info(run_id, experiment_name)['hidden_dimensions'])
print(get_run_info(run_id, experiment_name)['confidence_prob'])

[24]
0.9


In [5]:
output_seq.keys()

dict_keys([0, 1, 2, 3, 4])

In [6]:
for n_carries in output_seq.keys():
    print(n_carries)
    print(output_seq[n_carries].keys())
    print(len(output_seq[n_carries]['output_seq']))
    print(output_seq[n_carries]['input'].shape[0])
    print('-----')

0
dict_keys(['input', 'output', 'output_seq'])
30
81
-----
1
dict_keys(['input', 'output', 'output_seq'])
30
54
-----
2
dict_keys(['input', 'output', 'output_seq'])
30
52
-----
3
dict_keys(['input', 'output', 'output_seq'])
30
42
-----
4
dict_keys(['input', 'output', 'output_seq'])
30
27
-----


In [7]:
def decision_map(np_output, confidence_prob=0.9):
    unconfident_filter = np.greater(np_output, 1 - confidence_prob) * np.less(np_output, confidence_prob)
    np_decision = np_output * unconfident_filter
    np_decision = np_decision + np.greater_equal(np_output, confidence_prob)
    
    return np_decision

In [8]:
max_steps = len(output_seq[n_carries]['output_seq'])
#np.set_printoptions(precision=3)
count_dict = dict()
count_dict_no_max = dict()
count_dict_max = dict()
for n_carries in output_seq.keys():
    count_dict[n_carries] = list()
    count_dict_no_max[n_carries] = list()
    count_dict_max[n_carries] = 0
    print('[{} carries] =============================================================='.format(n_carries))
    for i in range(output_seq[n_carries]['input'].shape[0]):
        op1 = output_seq[n_carries]['input'][i][:4]
        op2 = output_seq[n_carries]['input'][i][4:]
        target = output_seq[n_carries]['output'][i]
        print('Operand1:', op1)
        print('Operand2:', op2)
        print('Target:', target)
        for i_step in range(max_steps):
            output = output_seq[n_carries]['output_seq'][i_step][i]
            decision = decision_map(output)
            
            if np.array_equal(target, decision):
                count_dict_no_max[n_carries].append(i_step)
                print(i_step, decision, '[Decision!]')
                break
            print(i_step, decision)
            if i_step == max_steps - 1:
                count_dict_max[n_carries] += 1
        count_dict[n_carries].append(i_step) 
        print('____________________________________________________________________')

[0 carries] ==============================================================
Operand1: [1 0 1 0]
Operand2: [0 1 0 1]
Target: [0 1 1 1 1]
0 [0.1474632  0.59237987 1.         1.         1.        ]
1 [0.        0.8836989 1.        1.        1.       ]
2 [0. 1. 1. 1. 1.] [Decision!]
____________________________________________________________________
Operand1: [1 0 0 0]
Operand2: [0 1 1 1]
Target: [0 1 1 1 1]
0 [0.         0.54087013 1.         1.         1.        ]
1 [0.         0.88542277 1.         1.         1.        ]
2 [0. 1. 1. 1. 1.] [Decision!]
____________________________________________________________________
Operand1: [0 1 1 0]
Operand2: [0 0 0 0]
Target: [0 0 1 1 0]
0 [0.         0.13867204 0.30137378 1.         0.        ]
1 [0.        0.        0.7896279 1.        0.       ]
2 [0. 0. 1. 1. 0.] [Decision!]
____________________________________________________________________
Operand1: [1 0 0 1]
Operand2: [0 0 1 0]
Target: [0 1 0 1 1]
0 [0.         0.37499252 0.1398351  1.   

In [7]:
for k, v in count_dict.items():
    print('{} carries: mean = {}'.format(k, np.mean(v)))

0 carries: mean = 1.3209876543209877
1 carries: mean = 1.7222222222222223
2 carries: mean = 2.1153846153846154
3 carries: mean = 2.0952380952380953
4 carries: mean = 2.5925925925925926


In [8]:
for k, v in count_dict_no_max.items():
    print('{} carries: mean = {} (n={})'.format(k, np.mean(v), len(v)))

0 carries: mean = 1.3209876543209877 (n=81)
1 carries: mean = 1.7222222222222223 (n=54)
2 carries: mean = 2.1153846153846154 (n=52)
3 carries: mean = 2.0952380952380953 (n=42)
4 carries: mean = 2.5925925925925926 (n=27)


In [9]:
for n_carries, v in count_dict_max.items():
    n_carry_total = output_seq[n_carries]['input'].shape[0]
    print('{} carries: max step count = {}, ratio = {}'.format(n_carries, v, round(v / n_carry_total, 2)))

0 carries: max step count = 0, ratio = 0.0
1 carries: max step count = 0, ratio = 0.0
2 carries: max step count = 0, ratio = 0.0
3 carries: max step count = 0, ratio = 0.0
4 carries: max step count = 0, ratio = 0.0
